In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
select current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE fhir_resources_stmnt STRING;

SET VARIABLE fhir_resources_stmnt = "
CREATE OR REFRESH STREAMING TABLE fhir_resources (
  resource_uuid STRING NOT NULL PRIMARY KEY COMMENT 'Uniquie identifier for the FHIR Resource in a bundle.'
  ,bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
  ,fullUrl STRING NOT NULL COMMENT 'The full URL of the resource in the entry array.  This serves as the primary key for the resource and later is used to join related resources from within a bundle.'
  ,resourceType STRING NOT NULL COMMENT 'The type of resource from the bundles entry array.'
  ,pos INT COMMENT 'The position of the resource element from within the specific resource itself.  Note that a bundle my have many of the same resources inside the entry array and positioning may not be consistent across resources or bundles.'
  ,key STRING NOT NULL COMMENT 'The name of the resources element extracted. These will serve as column names in the resource target tables.'
  ,value VARIANT COMMENT 'The value of the resource element extracted.  This will be a variant type and may contain nested variants.'
  ,CONSTRAINT fk_bundle_meta_bundle_uuid FOREIGN KEY (bundle_uuid) REFERENCES " || catalog_use || "." || schema_use || ".bundle_meta(bundle_uuid)
)
COMMENT 'Exploded FHIR Resources'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  sha2(concat(bundle_uuid, entry.value:fullUrl::string), 256) as resource_uuid
  ,bundle_uuid
  ,CAST(entry.value:fullUrl AS STRING) as fullUrl
  ,CAST(entry.value:resource.resourceType AS STRING) as resourceType
  ,resource.*
FROM 
  STREAM(fhir_bronze_variant)
  ,LATERAL variant_explode(fhir:entry) as entry
  ,LATERAL variant_explode(entry.value:resource) as resource
";

SELECT fhir_resources_stmnt;

In [0]:
EXECUTE IMMEDIATE fhir_resources_stmnt;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_resource_schemas (
  resourceType STRING COMMENT 'The type of resource from the bundles entry array.'
  ,column_name STRING COMMENT 'The name of the resources element extracted. These will serve as column names in the resource target tables.'
  ,schema_of_variant STRING COMMENT 'The schema of the resource element extracted as VARIANT.  The schema may be applied to the data elements to change into the elements official datatype.  Note that this is based on what was recieved in the bundles, which may include many nested extentions and is therefore more robust than schemas from the FHIR specefication documents.'
  ,schema_as_struct STRING COMMENT 'The same schema as the schema_of_variant, but with STRUCT instead of OBJECT. (Note that Matt is still determining if this is needed or not.)'
)
COMMENT 'Schemas of inferred from FHIR Resource Variant Data Types'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  resourceType
  ,key as column_name
  ,schema_of_variant_agg(value) as schema_of_variant
  ,REPLACE(schema_of_variant_agg(value), 'OBJECT', 'STRUCT') as schema_as_struct
FROM STREAM(fhir_resources)
GROUP BY 
  resourceType
  ,key